In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Abhinav\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Abhinav\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install --upgrade opencv-python

You should consider upgrading via the 'C:\Users\Abhinav\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [5]:
def load_images_from_folder(folder, label, img_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        img = cv2.imread(filepath)
        if img is not None:
            img = cv2.resize(img, img_size) # Resize image
            images.append(img)
            labels.append(label)
    return images, labels
fire_folder = "C:/Users/Abhinav/Desktop/Fire-Detection/fire_images"
non_fire_folder = "C:/Users/Abhinav/Desktop/Fire-Detection/non_fire_images"

In [6]:
# Load fire and non-fire images
fire_images, fire_labels = load_images_from_folder(fire_folder, label=1)
non_fire_images, non_fire_labels = load_images_from_folder(non_fire_folder, label=0)

In [7]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Fire color thresholds in HSV
lower_fire = np.array([0, 120, 200], dtype="uint8")
upper_fire = np.array([40, 255, 255], dtype="uint8")

def extract_features(image_path):
    """Extract fire-specific features: fire region ratio and mean red intensity."""
    image = cv2.imread(image_path)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_fire, upper_fire)

    # Calculate fire-like region ratio
    fire_area = np.sum(mask > 0)
    total_area = mask.size
    fire_ratio = fire_area / total_area

    # Calculate mean red channel intensity
    mean_red = np.mean(image[:, :, 2])

    return [fire_ratio, mean_red]

# Paths to dataset folders
fire_dir = "C:/Users/Abhinav/Desktop/Fire-Detection/fire_images"
non_fire_dir = "C:/Users/Abhinav/Desktop/Fire-Detection/non_fire_images"

X = []  # Features
y = []  # Labels

# Process fire images
for filename in os.listdir(fire_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        filepath = os.path.join(fire_dir, filename)
        X.append(extract_features(filepath))
        y.append(1)  # Fire label = 1

# Process non-fire images
for filename in os.listdir(non_fire_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        filepath = os.path.join(non_fire_dir, filename)
        X.append(extract_features(filepath))
        y.append(0)  # Non-fire label = 0

# Train a Random Forest model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save the trained model
joblib.dump(model, "fire_detection_model.pkl")
print("Model saved as 'fire_detection_model.pkl'")


Model Accuracy: 0.828125
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.94      0.90       102
           1       0.62      0.38      0.48        26

    accuracy                           0.83       128
   macro avg       0.74      0.66      0.69       128
weighted avg       0.81      0.83      0.81       128

Model saved as 'fire_detection_model.pkl'


In [8]:
pip install pyttsx3

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Abhinav\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
import tkinter as tk
from tkinter import filedialog, Label, Button
from PIL import Image, ImageTk
import cv2
import numpy as np
import joblib
import pyttsx3  # Text-to-speech library
import time 

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Load the trained Random Forest model
model = joblib.load("fire_detection_model.pkl")

# Fire color thresholds in HSV
lower_fire = np.array([0, 120, 200], dtype="uint8")
upper_fire = np.array([40, 255, 255], dtype="uint8")

def extract_features(image):
    """Extract fire-specific features from an image."""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_fire, upper_fire)

    # Fire-like region ratio
    fire_area = np.sum(mask > 0)
    total_area = mask.size
    fire_ratio = fire_area / total_area

    # Mean red channel intensity
    mean_red = np.mean(image[:, :, 2])

    return [fire_ratio, mean_red]

def speak(text):
    """Speak the given text using pyttsx3."""
    
    engine.say(text)
    engine.runAndWait()

def load_image():
    """Load an image, extract features, and predict fire presence."""
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png")])
    if file_path:
        # Read and display the image
        image = cv2.imread(file_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_pil = Image.fromarray(image_rgb)
        image_tk = ImageTk.PhotoImage(image_pil)
        label_img.config(image=image_tk)
        label_img.image = image_tk

        # Extract features and predict fire
        features = extract_features(image)
        prediction = model.predict([features])[0]
        
        if prediction == 1:
            result_text = "Fire Detected!"
            result_label.config(text=result_text, fg="red")
            time.sleep(5) 
            speak("Warning! Fire has been detected.")
        else:
            result_text = "No Fire Detected"
            result_label.config(text=result_text, fg="green")
            time.sleep(5) 
            speak("No fire detected. The area is safe.")

# GUI Application
root = tk.Tk()
root.title("Fire Detection System")
root.geometry("600x400")
root.configure(bg="white")

# Title
title = Label(root, text="Fire Detection System", font=("Helvetica", 16, "bold"), bg="white")
title.pack(pady=10)

# Image Display Area
label_img = Label(root, bg="white")
label_img.pack(pady=10)

# Result Label
result_label = Label(root, text="", font=("Helvetica", 14), bg="white")
result_label.pack(pady=10)

# Buttons
btn_load = Button(root, text="Load Image", command=load_image, font=("Helvetica", 12), bg="blue", fg="white")
btn_load.pack(pady=10)

# Run Application
root.mainloop()
